<a href="https://colab.research.google.com/github/planewave/coherent_receiver_with_CNN/blob/master/CNN_for_symbol_timing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 -q install torch
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
print(torch.__version__)
USE_CUDA = True

tcmalloc: large alloc 1073750016 bytes == 0x58c48000 @  0x7f3c822952a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
0.4.1


In [0]:
def rrcosdesign(beta, span, sps):
    """
    returns the coefficients, that correspond to a square-root raised 
    cosine FIR filter with rolloff factor specified by beta. The filter 
    is truncated to span symbols, and each symbol period contains sps 
    samples. The order of the filter, sps*span, must be even. 
    modified from:
    https://github.com/veeresht/CommPy/blob/master/commpy/filters.py
    """
    n = sps*span
    rrc = np.zeros(n, dtype=float)
        
    for x in np.arange(n):
        t = (x-n/2)/sps
        if t == 0.0:
            rrc[x] = 1.0 - beta + (4*beta/np.pi)
        elif beta != 0 and t == 1/(4*beta):
            rrc[x] = ((beta/np.sqrt(2))*(((1+2/np.pi)*(np.sin(np.pi/
                     (4*beta)))) + ((1-2/np.pi)*(np.cos(np.pi/(4*beta))))))
        elif beta != 0 and t == -1/(4*beta):
            rrc[x] = ((beta/np.sqrt(2))*(((1+2/np.pi)*(np.sin(np.pi/
                     (4*beta)))) + ((1-2/np.pi)*(np.cos(np.pi/(4*beta))))))
        else:
            rrc[x] = ((np.sin(np.pi*t*(1-beta))+4*beta*(t)*np.cos(np.pi*
                     t*(1+beta)))/(np.pi*t*(1-(4*beta*t)*(4*beta*t))))
        
    return rrc

def upsample(x, n):
    """
    increase sample rate by integer factor
    y = upsample(x,n) increases the sample rate of x by 
    inserting n – 1 zeros between samples.
    input is 1D numpy array
    
    """
    zo = np.zeros((len(x), n), dtype=x.dtype)
    zo[:,0] += x
    return zo.flatten()

In [63]:
def gen_sig():
    M = 4
    SYMB = 16  # num of symbols
    BATCH = 256
    beta, span, sps = 0.4, 6, 16
    rrc = rrcosdesign(beta, span, sps)
#     sig_batch = np.array([])
    msg = np.random.randint(0, M, SYMB*BATCH)
    sig_mod = np.exp(1j*(np.pi/M+msg*(2*np.pi/M)))
    sig_up = upsample(sig_mod, sps)  # SYMB*sps (512,)
    sig_pulse = np.convolve(sig_up, rrc)[sps*3:-sps*3+1]
    sig = sig_pulse.reshape((BATCH, 1, SYMB*sps))
    offset = np.random.randint(-6, 6, BATCH)
    for ind in range(BATCH):
        sig[ind, :, :] = np.roll(sig[ind, :, :], offset(ind))
#     msg = msg.reshape((BATCH, 1, SYMB))
    return sig, offset       

sig, msg = gen_sig()   
sig.shape
# plt.plot(sig.imag, '-o')
# msg

(256, 1, 256)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.sto_cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv2d(32, 16, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv2d(16, 8, 3, 1, 1),
            nn.ReLU(),
         
        self.lin = nn.Linear(32, 1)    
        )
    def forward(self, sig):
        out_cnn = self.sto_cnn(sig)
        out_mean = out_cnn.squeeze().mean(2)
        return nn.functional.softmax(out_mean, dim=1)
    
net = Net()
if USE_CUDA:
    offset = offset.cuda()
    sig = sig.cuda()
    net.cuda() # put it before optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

# Training

In [0]:
for epoch in range(NUM_EPO):
    netout = net(sig)
    target = offset
    loss = loss_func(netout, target)
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()
    if epoch%5 == 0:
        prediction = torch.max(F.softmax(netout, 1), 1)[1]
        accu = torch.sum(prediction.data == target.data)/prediction.shape[0]
        print('Epoch:', epoch, 'loss:', loss, 'accu', accu)


In [13]:
sig = (dataRead['x'])
sig = Variable(torch.from_numpy(sig))
m = nn.Sequential(
            nn.Conv2d( 1, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16,  4, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d( 4,  1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
        )
output = m(sig)
output = output.squeeze().mean(2)
output.shape

# m2 = nn.Softmax(dim=1)
# output = m2(output)
# output[0,:]
# out_cnn = self.sto_cnn(sig)
# out_mean = out_cnn.squeeze().mean(2)
netout = nn.functional.softmax(output, dim=1)
prediction = torch.max(F.softmax(netout, 1), 1)[1]
offset = dataRead['offset']
target = Variable(torch.from_numpy(offset).squeeze())
accu = torch.sum(prediction.data == target.data)/prediction.shape[0]
accu

# print(target)
loss_func = nn.CrossEntropyLoss()

loss = loss_func(netout, target)
loss

Variable containing:
 2.7727
[torch.FloatTensor of size 1]